# Import Dependencies

In [1]:
# Import Dependencies
import os, re, time, torch, ffmpeg, whisper
import pandas as pd
from pytubefix import YouTube, Stream
from pytubefix.cli import on_progress
from pytubefix.innertube import _default_clients
from tqdm.auto import tqdm
from IPython.display import clear_output

# Define Utilities

In [8]:
def sanitize_filename(filename: str) -> str:
    # Escape Double Quotes
    filename = filename.replace('"', '\\"')

    # Replace Invalid Characters with "_"
    invalid_chars = re.compile(r'[<>:"/\\|?*]')
    sanitized_filename = invalid_chars.sub("_", filename)

    return sanitized_filename

def read_unique_items_from_csv(file: str, column: str = "URL") -> list:
    if os.path.exists(file):
        with open(file, "r", errors="ignore") as file:
            return pd.read_csv(file)[column].tolist()
    return []

def get_yt_id(url: str):
    # Find the part after the "?" (query parameters)
    if "?" in url: query_string = url.split("?")[-1]
    else: return None
    # Split the query parameters by "&"
    params = query_string.split("&")
    # Find the parameter by name
    for param in params:
        key, value = param.split("=")
        if key == "v": return value
    return None

# Set Configurations

In [ ]:
# File Names
yt_video_links_filename = os.path.join("..", "1) Collecting Video URLs (YouTube)", "YouTube URL Collection", "Used URLs.csv")

# Folder Names
video_output_path = "Video"
audio_output_path = "Audio"
transcription_output_path = "Transcription"

# Boolean Flags
run_continuously = True
remove_video = True
remove_audio = True
authorize_yt = False # Change to true for auth if YouTube Detects you as a BOT

# Additional Dependency Configurations
_default_clients["ANDROID_MUSIC"] = _default_clients["ANDROID_CREATOR"]

# Print Colors
class pcolors:
    ENDC = "\033[0m"
    BRIGHTCYAN = "\033[38;2;67;227;235m"
    BRIGHTRED = "\033[38;2;232;0;0m"

# Collect Data (YouTube Videos)

In [10]:
def download_youtube_video(video_filename: str, stream: Stream) -> tuple:
    # Create Video Directory
    os.makedirs(video_output_path, exist_ok=True)
    
    # Set Path for Video File
    video_file = os.path.join(video_output_path, video_filename)
    
    # Delete Old Existing Video File (note: to clean any corrupted file)
    if os.path.exists(video_file):
        os.remove(video_file)
        
    # Download Video File
    print("") # Just New Line for Better Output
    print(f'Downloading (Video): {video_filename}')
    print("") # Just New Line for Better Output
    stream.download(output_path=video_output_path, filename=video_filename)
    print("") # Just New Line for Better Output
    print("") # Just New Line for Better Output
    
    # Return Video File and Name
    return video_file, video_filename

# Audio Extraction (Video to Audio)

In [11]:
def extract_audio_from_video(video_file: str, video_filename: str) -> tuple:
    # Create the Audio Directory
    os.makedirs(audio_output_path, exist_ok=True)

    # Set Audio File Name ("[YouTube Video ID] [title].mp3")
    audio_filename = f'{os.path.splitext(video_filename)[0]}.mp3'

    # Set Path for Audio File
    audio_file = os.path.join(audio_output_path, audio_filename)
    
    # Delete Old Existing Audio File (note: to clean any corrupted file)
    if os.path.exists(audio_file):
        os.remove(audio_file)
    
    # Extract Audio File
    print(f'Extracting (Audio): {audio_filename}')
    print("") # Just New Line for Better Output
    (
        ffmpeg
        .input(video_file)
        .output(audio_file, format="mp3", acodec="libmp3lame", loglevel="info")
        .run(overwrite_output=True)
    )
    
    # Return Audio File and Name
    return audio_file, audio_filename

# Transcription (Audio to Text)

In [12]:
def transcribe_audio_to_text(audio_file: str, audio_filename: str):
    # Create the Transcription Directory
    os.makedirs(transcription_output_path, exist_ok=True)
    
    # Set Transcription File Name ("[YouTube Video ID] [title].txt")
    transcription_filename = f'{os.path.splitext(audio_filename)[0]}.txt'
    
    # Set Path for Transcription File
    transcription_file = os.path.join(transcription_output_path, transcription_filename)
            
    # Get/Download OpenAI Whisper Model
    """ 
    Models: 
        tiny, base, small, medium, large, turbo
    English-Only:
        tiny.en, base.en, small.en, medium.en
    
    Required VRAM:              Speed:
        1) 1GB - tiny, base         1) 10x - tiny
        2) 2GB - small              2) 8x - turbo
        3) 5GB - medium             3) 7x - base
        4) 6GB - turbo              4) 4x - small
        5) 10GB - large             5) 2x - medium
                                    6) 1x - large
    
    Quote from OpenAI: 
        - The .en models for English-only applications tend to perform better, especially for the tiny.en and base.en models.
        We observed that the difference becomes less significant for the small.en and medium.en models.
    
    Note: Laptop has 4GB VRAM so small.en is used
    """  
    print(f'Transcribing (Text): {transcription_filename}')
    print("") # Just New Line for Better Output
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
    model = whisper.load_model("small.en", device=device)
    
    # Transcribe Audio File (Saves Whole Text in Memory Before Disk to Avoid Corruption)
    result = model.transcribe(audio_file, fp16=False, verbose=False)
    try:
        with open(transcription_file, "w", errors="ignore") as f:
            f.write(result["text"])
    except:
        if os.path.exists(transcription_file):
            os.remove(transcription_file)

# Execute Data Gathering

In [ ]:
while True:
    try:
        yt_urls = read_unique_items_from_csv(yt_video_links_filename)
        with tqdm(total=len(yt_urls), desc="Getting YouTube URLs") as pbar:
            for index, url in enumerate(yt_urls):
                try:
                    current = f'{index+1}/{len(yt_urls)}'
                    
                    # Get YouTube Id
                    video_id = get_yt_id(url)

                    # Skip If No YouTube Video Id Found in URL
                    if not video_id:
                        print("") # Just New Line for Better Output
                        print(f'Invalid YouTube URL: {url}')
                        
                        pbar.update(1)
                        continue
                    
                    # Set Filename Video Id Pattern
                    video_id_pattern_in_filename = re.compile(r'\[(.*?)\]')
                    
                    # Skip If Transcription with YouTube ID Already Exists
                    has_transcription_file = False
                    pbar.set_description(f'Checking Existing Transcriptions [{current} File]')
                    if os.path.exists(transcription_output_path):
                        for existing_transcription_filename in os.listdir(transcription_output_path):
                            if (
                                existing_transcription_filename == ".ipynb_checkpoints"
                                or existing_transcription_filename == "Extra (not included in any [used urls.csv])"
                            ): continue
                            if (
                                video_id_pattern_in_filename.search(existing_transcription_filename)
                                and video_id_pattern_in_filename.search(existing_transcription_filename).group(1) == video_id
                            ): 
                                has_transcription_file = True
                                break
                    if has_transcription_file:
                        # Delete/Keep Video File
                        if remove_video:
                            for existing_video_filename in os.listdir(video_output_path):
                                if existing_video_filename == ".ipynb_checkpoints": continue
                                if (
                                    video_id_pattern_in_filename.search(existing_video_filename)
                                    and video_id_pattern_in_filename.search(existing_video_filename).group(1) == video_id
                                ): os.remove(existing_video_filename)
                                    
                        # Delete/Keep Audio File
                        if remove_audio:
                            for existing_audio_filename in os.listdir(audio_output_path):
                                if existing_audio_filename == ".ipynb_checkpoints": continue
                                if (
                                    video_id_pattern_in_filename.search(existing_audio_filename)
                                    and video_id_pattern_in_filename.search(existing_audio_filename).group(1) == video_id
                                ): os.remove(existing_audio_filename)
                                    
                        pbar.update(1)
                        continue

                    # Get Video Information
                    yt = YouTube(
                        url,
                        use_oauth=authorize_yt,
                        allow_oauth_cache=authorize_yt,
                        on_progress_callback=on_progress
                    )
                    stream = yt.streams.get_audio_only()

                    # Sanitize Video File Name and Add YouTube Video ID
                    video_filename = f'[{video_id}] {sanitize_filename(stream.default_filename)}'

                    # Get File Name Without Extension (e.g., ".mp4")
                    filename = os.path.splitext(video_filename)[0]

                    # Log YouTube URL being Processed
                    print("") # Just New Line for Better Output
                    print(f'Found YouTube Video (URL): {url}')

                    # Download YouTube Video
                    pbar.set_description(f'Downloading [{current} Video] ')
                    video_file, video_filename = download_youtube_video(video_filename, stream)

                    # Extract Audio from Video -> Delete/Keep Video File
                    pbar.set_description(f'Extracting [{current} Audio]')
                    audio_file, audio_filename = extract_audio_from_video(video_file, video_filename)
                    if remove_video: os.remove(video_file)

                    # Transcribe Audio to Text -> Delete/Keep Audio File
                    pbar.set_description(f'Transcribing [{current} Text]')
                    transcribe_audio_to_text(audio_file, audio_filename)
                    if remove_audio: os.remove(audio_file)

                    pbar.update(1)
                except Exception as e: 
                    print(f'{e}: {url}')
                    
                    pbar.update(1)
            pbar.set_description("Finished Data Gathering")
    except Exception as e: print(e)
    # If Set To Run Endlessly Until User Interruption
    if run_continuously:
        clear_output(wait=True)
        print("") # Just New Line for Better Output
        print(f'{pcolors.BRIGHTCYAN}... This Stays Alive ...{pcolors.ENDC}')
        print("") # Just New Line for Better Output
        print(f'{pcolors.BRIGHTCYAN}... You Can Now Add More URL ...{pcolors.ENDC}')
        print("") # Just New Line for Better Output
        print(f'{pcolors.BRIGHTCYAN}... This Will Automatically Recheck and Transcribe New Added URLs ...{pcolors.ENDC}')
        print("") # Just New Line for Better Output
        print(f'{pcolors.BRIGHTRED}... [ Click Interrupt To Stop ] ...{pcolors.ENDC}')
        print("") # Just New Line for Better Output
        time.sleep(5)
        continue
    break


Found YouTube Video (URL): https://www.youtube.com/watch?v=CcsRtLQMSmg


 ↳ |██████████████████████████████████████████████████████████████████| 100.0%

Extracting [1/22 Audio]:   0%|                                                                  | 0/22 [00:02<?, ?it/s]



Extracting (Audio): [CcsRtLQMSmg] JD Vance_ 'There Are 453,000 Illegal Immigrants' In AZ 'Because Kamala Harris Refuses To Do Her Job'.mp3



Transcribing [1/22 Text]:   0%|                                                                 | 0/22 [00:05<?, ?it/s]

Transcribing (Text): [CcsRtLQMSmg] JD Vance_ 'There Are 453,000 Illegal Immigrants' In AZ 'Because Kamala Harris Refuses To Do Her Job'.txt



...

Extracting [21/22 Audio]:  91%|████████████████████████████████████████████████▏    | 20/22 [1:07:47<12:47, 383.58s/it]



Extracting (Audio): [Sz7-TlodhSA] Kamala Harris Arizona Rally Key Takeaways.mp3



Transcribing [21/22 Text]:  91%|███████████████████████████████████████████████▎    | 20/22 [1:07:50<12:47, 383.58s/it]

Transcribing (Text): [Sz7-TlodhSA] Kamala Harris Arizona Rally Key Takeaways.txt




 65%|██████████████████████████████████████████████                         | 21432/33048 [00:46<00:25, 454.57frames/s]